# 🏦 Nigerian Bank Transaction Fraud Detection
## End-to-End Data Science Project

---

**Author:** Data Science Portfolio Project  
**Domain:** Financial Technology (FinTech) | Fraud Analytics  
**Tools:** Python, Pandas, Scikit-learn, Matplotlib, Seaborn

---

## 📌 Project Overview

Financial fraud is a critical challenge for Nigerian banks. According to the Financial Institutions Training Centre (FITC), Nigerian banks lost over **₦9.5 billion** to fraud in 2023 alone. This project builds a **Machine Learning-powered fraud detection system** that can automatically flag suspicious transactions in real-time, helping banks like GTBank, Access Bank, Zenith Bank, and UBA protect their customers.

## 🎯 Business Objectives

1. **Detect** fraudulent transactions with high accuracy before they are processed
2. **Reduce** financial losses to the bank and customers
3. **Minimize** false positives (blocking genuine customers)
4. **Provide** interpretable insights for the fraud investigation team
5. **Quantify** the business impact of deploying the model

## 📋 Project Workflow

```
1. Data Simulation       → Generate realistic Nigerian bank transactions
2. Exploratory Analysis  → Understand patterns, distributions, fraud indicators
3. Data Cleaning         → Handle missing values, outliers, feature engineering
4. Anomaly Detection     → Statistical methods to flag suspicious transactions
5. ML Classification     → Train & evaluate fraud detection models
6. Business Impact       → Translate model performance into ₦ savings
```

---
# SECTION 1: Environment Setup & Library Imports
---

In [ ]:
# ============================================================
# IMPORT LIBRARIES
# ============================================================

# Core data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    roc_curve, precision_recall_curve, average_precision_score,
    f1_score, precision_score, recall_score
)
from sklearn.inspection import permutation_importance

# Utilities
import warnings
import random
from datetime import datetime, timedelta

warnings.filterwarnings('ignore')
np.random.seed(42)
random.seed(42)

# Visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
COLORS = {'fraud': '#E74C3C', 'legit': '#2ECC71', 'neutral': '#3498DB', 'accent': '#F39C12'}

print('✅ All libraries imported successfully!')
print(f'   NumPy version  : {np.__version__}')
print(f'   Pandas version : {pd.__version__}')

---
# SECTION 2: Data Simulation
### Generating Realistic Nigerian Bank Transaction Data
---

Since real bank data is confidential, we simulate transactions using domain knowledge of Nigerian banking patterns:
- **NIBSS** (Nigeria Inter-Bank Settlement System) transaction types
- Typical Nigerian salary ranges and spending patterns
- Common fraud vectors: ATM skimming, POS fraud, mobile banking takeover, BVN fraud

In [ ]:
# ============================================================
# DATA SIMULATION: Nigerian Bank Transactions
# ============================================================

def simulate_nigerian_bank_data(n_transactions=50000, fraud_rate=0.025):
    """
    Simulates realistic Nigerian bank transaction data.
    
    Fraud rate ~2.5% reflects industry estimates from FITC Nigeria reports.
    Features are designed to mirror real NIBSS transaction attributes.
    """
    
    np.random.seed(42)
    n_fraud = int(n_transactions * fraud_rate)
    n_legit = n_transactions - n_fraud
    
    # --- Nigerian Geographic & Banking Context ---
    states = ['Lagos', 'Abuja', 'Rivers', 'Kano', 'Oyo', 'Delta', 'Anambra',
              'Kaduna', 'Ogun', 'Edo', 'Imo', 'Enugu', 'Cross River', 'Plateau']
    
    channels = ['Mobile App', 'ATM', 'POS', 'Internet Banking', 'USSD', 'Branch']
    channel_weights = [0.35, 0.20, 0.22, 0.12, 0.08, 0.03]  # Mobile is dominant in Nigeria
    
    transaction_types = ['Transfer', 'Withdrawal', 'Payment', 'Airtime', 'Utility Bill',
                         'Salary Credit', 'Card Purchase', 'International Transfer']
    
    merchant_categories = ['Supermarket', 'Fuel Station', 'Restaurant', 'Hospital',
                            'Telecom', 'Utilities', 'Electronics', 'Fashion', 
                            'Betting', 'Online Merchant', 'Money Transfer', 'ATM']
    
    # ============================================================
    # GENERATE LEGITIMATE TRANSACTIONS
    # ============================================================
    
    # Transaction timestamps — business hours weighted higher
    start_date = datetime(2023, 1, 1)
    legit_raw = np.array([1,1,1,1,1,2,3,6,7,8,7,7,7,7,7,6,5,5,4,4,3,3,2,1], dtype=float)
    legit_hour_p = legit_raw / legit_raw.sum()
    legit_hours = np.random.choice(
        range(24),
        size=n_legit,
        p=legit_hour_p
    )
    legit_days = np.random.randint(0, 365, n_legit)
    
    # Transaction amounts — Nigerian Naira, log-normal distribution
    # Average Nigerian salary ~₦100,000–₦400,000/month
    legit_amounts = np.random.lognormal(mean=10.5, sigma=1.8, size=n_legit)
    legit_amounts = np.clip(legit_amounts, 100, 5_000_000)  # ₦100 to ₦5M
    
    # Customer profiles
    legit_account_age_days = np.random.gamma(shape=5, scale=200, size=n_legit)  # avg 1000 days
    legit_avg_balance = np.random.lognormal(mean=11.5, sigma=1.5, size=n_legit)
    legit_txn_count_30d = np.random.poisson(lam=20, size=n_legit)  # avg 20 txns/month
    legit_txn_count_7d = np.random.poisson(lam=5, size=n_legit)
    legit_distance_from_home = np.random.exponential(scale=15, size=n_legit)  # km
    legit_failed_attempts = np.random.choice([0, 1, 2], size=n_legit, p=[0.92, 0.06, 0.02])
    legit_is_new_recipient = np.random.choice([0, 1], size=n_legit, p=[0.78, 0.22])
    
    # ============================================================
    # GENERATE FRAUDULENT TRANSACTIONS
    # ============================================================
    
    # Fraudsters prefer late night / early morning
    fraud_raw = np.array([5,6,7,7,6,5,4,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,5], dtype=float)
    fraud_hour_p = fraud_raw / fraud_raw.sum()
    fraud_hours = np.random.choice(
        range(24),
        size=n_fraud,
        p=fraud_hour_p
    )
    fraud_days = np.random.randint(0, 365, n_fraud)
    
    # Fraud amounts — either very small (testing) or very large (drain)
    fraud_amount_type = np.random.choice(['small', 'large'], size=n_fraud, p=[0.3, 0.7])
    fraud_amounts = np.where(
        fraud_amount_type == 'small',
        np.random.uniform(50, 1000, n_fraud),          # Micro-probing transactions
        np.random.lognormal(mean=13, sigma=1.2, size=n_fraud)  # Large draining
    )
    fraud_amounts = np.clip(fraud_amounts, 50, 50_000_000)
    
    # Fraudulent account characteristics
    fraud_account_age_days = np.random.gamma(shape=1.5, scale=100, size=n_fraud)  # Newer accounts
    fraud_avg_balance = np.random.lognormal(mean=10, sigma=1.2, size=n_fraud)
    fraud_txn_count_30d = np.random.poisson(lam=40, size=n_fraud)   # Unusual spike
    fraud_txn_count_7d = np.random.poisson(lam=12, size=n_fraud)    # High recent activity
    fraud_distance_from_home = np.random.exponential(scale=200, size=n_fraud)  # Far from home
    fraud_failed_attempts = np.random.choice([0, 1, 2, 3], size=n_fraud, p=[0.40, 0.25, 0.20, 0.15])
    fraud_is_new_recipient = np.random.choice([0, 1], size=n_fraud, p=[0.20, 0.80])  # New recipients
    
    # ============================================================
    # COMBINE & BUILD DATAFRAME
    # ============================================================
    
    all_hours   = np.concatenate([legit_hours, fraud_hours])
    all_days    = np.concatenate([legit_days, fraud_days])
    all_amounts = np.concatenate([legit_amounts, fraud_amounts])
    all_labels  = np.concatenate([np.zeros(n_legit), np.ones(n_fraud)])
    
    # Shuffle everything together
    shuffle_idx = np.random.permutation(n_transactions)
    
    all_accounts = [f'NG{np.random.randint(10_000_000, 99_999_999)}' for _ in range(n_transactions)]
    
    df = pd.DataFrame({
        'transaction_id'       : [f'TXN{i:08d}' for i in range(n_transactions)],
        'account_number'       : all_accounts,
        'transaction_date'     : [start_date + timedelta(days=int(d)) for d in all_days[shuffle_idx]],
        'transaction_hour'     : all_hours[shuffle_idx],
        'amount_ngn'           : np.round(all_amounts[shuffle_idx], 2),
        'transaction_type'     : np.random.choice(transaction_types, n_transactions),
        'channel'              : np.random.choice(channels, n_transactions, p=channel_weights),
        'merchant_category'    : np.random.choice(merchant_categories, n_transactions),
        'state'                : np.random.choice(states, n_transactions),
        'account_age_days'     : np.concatenate([
                                    legit_account_age_days, fraud_account_age_days
                                 ])[shuffle_idx].round(0),
        'avg_balance_ngn'      : np.concatenate([
                                    legit_avg_balance, fraud_avg_balance
                                 ])[shuffle_idx].round(2),
        'txn_count_last_30d'   : np.concatenate([
                                    legit_txn_count_30d, fraud_txn_count_30d
                                 ])[shuffle_idx],
        'txn_count_last_7d'    : np.concatenate([
                                    legit_txn_count_7d, fraud_txn_count_7d
                                 ])[shuffle_idx],
        'distance_from_home_km': np.concatenate([
                                    legit_distance_from_home, fraud_distance_from_home
                                 ])[shuffle_idx].round(2),
        'failed_attempts_24h'  : np.concatenate([
                                    legit_failed_attempts, fraud_failed_attempts
                                 ])[shuffle_idx],
        'is_new_recipient'     : np.concatenate([
                                    legit_is_new_recipient, fraud_is_new_recipient
                                 ])[shuffle_idx],
        'is_fraud'             : all_labels[shuffle_idx].astype(int)
    })
    
    # Inject ~3% missing values in non-critical fields (realistic data quality)
    for col in ['merchant_category', 'distance_from_home_km', 'avg_balance_ngn']:
        missing_mask = np.random.choice([True, False], size=len(df), p=[0.03, 0.97])
        df.loc[missing_mask, col] = np.nan
    
    return df


# Generate the dataset
df_raw = simulate_nigerian_bank_data(n_transactions=50000, fraud_rate=0.025)

print('=' * 60)
print('  ✅ DATASET GENERATED SUCCESSFULLY')
print('=' * 60)
print(f'  Total Transactions : {len(df_raw):,}')
print(f'  Fraudulent         : {df_raw["is_fraud"].sum():,} ({df_raw["is_fraud"].mean()*100:.1f}%)')
print(f'  Legitimate         : {(df_raw["is_fraud"]==0).sum():,} ({(df_raw["is_fraud"]==0).mean()*100:.1f}%)')
print(f'  Features           : {df_raw.shape[1] - 1}')
print(f'  Date Range         : {df_raw["transaction_date"].min().date()} to {df_raw["transaction_date"].max().date()}')
print('=' * 60)

In [ ]:
# Preview the first few rows
print('📋 DATASET PREVIEW:')
df_raw.head(10)

In [ ]:
# Dataset structure and data types
print('🗂️  DATASET INFO:')
df_raw.info()
print('\n📊 STATISTICAL SUMMARY:')
df_raw.describe()

---
# SECTION 3: Exploratory Data Analysis (EDA)
### Understanding Patterns, Distributions & Fraud Signals
---

In [ ]:
# ============================================================
# 3.1 — CLASS DISTRIBUTION (Fraud vs Legitimate)
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
labels = ['Legitimate', 'Fraudulent']
counts = df_raw['is_fraud'].value_counts().sort_index()
bars = axes[0].bar(labels, counts, color=[COLORS['legit'], COLORS['fraud']], 
                   edgecolor='white', linewidth=2, width=0.5)
axes[0].set_title('Transaction Class Distribution', fontsize=14, fontweight='bold', pad=15)
axes[0].set_ylabel('Number of Transactions', fontsize=12)
for bar, count in zip(bars, counts):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 200,
                f'{count:,}', ha='center', va='bottom', fontsize=12, fontweight='bold')
axes[0].yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, p: f'{x:,.0f}'))

# Pie chart
axes[1].pie(counts, labels=labels, colors=[COLORS['legit'], COLORS['fraud']],
            autopct='%1.1f%%', startangle=90, textprops={'fontsize': 12},
            wedgeprops={'edgecolor': 'white', 'linewidth': 2})
axes[1].set_title('Fraud Rate Distribution', fontsize=14, fontweight='bold', pad=15)

plt.suptitle('⚠️ Dataset is Imbalanced — A Key Challenge in Fraud Detection', 
             fontsize=13, y=1.02, color='gray', style='italic')
plt.tight_layout()
plt.show()

print(f"""
💡 KEY INSIGHT: Class Imbalance
─────────────────────────────────────────────────────────────
Only {df_raw['is_fraud'].mean()*100:.1f}% of transactions are fraud. This is realistic but
creates a challenge: a naive model that labels EVERYTHING as
legitimate would achieve {(1-df_raw['is_fraud'].mean())*100:.1f}% accuracy — but catch ZERO fraud!

Strategy: Use class_weight='balanced' in models + prioritize
Recall and ROC-AUC over simple accuracy.
─────────────────────────────────────────────────────────────
""")

In [ ]:
# ============================================================
# 3.2 — TRANSACTION AMOUNT ANALYSIS
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

fraud_amounts  = df_raw[df_raw['is_fraud'] == 1]['amount_ngn']
legit_amounts  = df_raw[df_raw['is_fraud'] == 0]['amount_ngn']

# Log-scale distribution
axes[0].hist(np.log10(legit_amounts + 1), bins=60, alpha=0.7,
             color=COLORS['legit'], label=f'Legitimate (n={len(legit_amounts):,})', density=True)
axes[0].hist(np.log10(fraud_amounts + 1), bins=60, alpha=0.7,
             color=COLORS['fraud'], label=f'Fraud (n={len(fraud_amounts):,})', density=True)
axes[0].set_title('Transaction Amount Distribution (Log₁₀ Scale)', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Log₁₀(Amount in ₦)', fontsize=12)
axes[0].set_ylabel('Density', fontsize=12)
axes[0].legend(fontsize=11)

# Nigerian Naira x-tick labels
tick_values = [2, 3, 4, 5, 6, 7]
tick_labels = ['₦100', '₦1K', '₦10K', '₦100K', '₦1M', '₦10M']
axes[0].set_xticks(tick_values)
axes[0].set_xticklabels(tick_labels, rotation=30)

# Box plot comparison
amount_data = [
    np.log10(legit_amounts.clip(lower=1) + 1),
    np.log10(fraud_amounts.clip(lower=1) + 1)
]
bp = axes[1].boxplot(amount_data, labels=['Legitimate', 'Fraudulent'],
                    patch_artist=True, notch=True)
bp['boxes'][0].set_facecolor(COLORS['legit'])
bp['boxes'][1].set_facecolor(COLORS['fraud'])
axes[1].set_title('Amount Distribution — Box Plot Comparison', fontsize=13, fontweight='bold')
axes[1].set_ylabel('Log₁₀(Amount in ₦)', fontsize=12)
axes[1].set_yticks(tick_values)
axes[1].set_yticklabels(tick_labels)

plt.suptitle('🔍 Fraud vs Legitimate Transaction Amounts in Nigerian Naira (₦)',
             fontsize=13, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

print(f"""
📊 AMOUNT STATISTICS (Nigerian Naira ₦)
─────────────────────────────────────────────────────────────
                    Legitimate          Fraudulent
  Median Amount  :  ₦{legit_amounts.median():>12,.0f}    ₦{fraud_amounts.median():>12,.0f}
  Mean Amount    :  ₦{legit_amounts.mean():>12,.0f}    ₦{fraud_amounts.mean():>12,.0f}
  Max Amount     :  ₦{legit_amounts.max():>12,.0f}    ₦{fraud_amounts.max():>12,.0f}
─────────────────────────────────────────────────────────────
💡 Fraud transactions tend to be larger on average — 
   fraudsters maximize their take per successful attempt.
""")

In [ ]:
# ============================================================
# 3.3 — TIME-BASED FRAUD ANALYSIS
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(18, 12))

legit_hour_p = np.array([1,1,1,1,1,2,3,6,7,8,7,7,7,7,7,6,5,5,4,4,3,3,2,1], dtype=float)
legit_hour_p /= legit_hour_p.sum()
fraud_hour_p = np.array([5,6,7,7,6,5,4,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,5], dtype=float)
fraud_hour_p /= fraud_hour_p.sum()

# --- Fraud rate by hour of day ---
hourly = df_raw.groupby('transaction_hour')['is_fraud'].agg(['mean', 'count'])
ax = axes[0, 0]
bars = ax.bar(hourly.index, hourly['mean'] * 100,
              color=[COLORS['fraud'] if h >= 22 or h <= 5 else COLORS['neutral'] 
                     for h in hourly.index])
ax.set_title('🕐 Fraud Rate by Hour of Day', fontsize=13, fontweight='bold')
ax.set_xlabel('Hour (24-hour format)', fontsize=11)
ax.set_ylabel('Fraud Rate (%)', fontsize=11)
ax.axvspan(-0.5, 5.5, alpha=0.1, color='red', label='High-risk: 00:00–06:00')
ax.axvspan(21.5, 23.5, alpha=0.1, color='red')
ax.legend(fontsize=10)
ax.set_xticks(range(0, 24, 2))
ax.set_xticklabels([f'{h:02d}:00' for h in range(0, 24, 2)], rotation=30)

# --- Fraud rate by transaction type ---
type_fraud = df_raw.groupby('transaction_type')['is_fraud'].mean().sort_values(ascending=True)
ax = axes[0, 1]
bars = ax.barh(type_fraud.index, type_fraud.values * 100,
               color=[COLORS['fraud'] if v > type_fraud.mean() else COLORS['neutral'] 
                      for v in type_fraud.values])
ax.set_title('💳 Fraud Rate by Transaction Type', fontsize=13, fontweight='bold')
ax.set_xlabel('Fraud Rate (%)', fontsize=11)
ax.axvline(type_fraud.mean() * 100, color='red', linestyle='--', alpha=0.7, label='Average')
ax.legend(fontsize=10)

# --- Fraud rate by channel ---
channel_fraud = df_raw.groupby('channel')['is_fraud'].mean().sort_values(ascending=False)
ax = axes[1, 0]
colors = [COLORS['fraud'] if v > channel_fraud.mean() else COLORS['legit'] 
          for v in channel_fraud.values]
ax.bar(channel_fraud.index, channel_fraud.values * 100, color=colors, edgecolor='white', linewidth=1.5)
ax.set_title('📱 Fraud Rate by Banking Channel', fontsize=13, fontweight='bold')
ax.set_ylabel('Fraud Rate (%)', fontsize=11)
ax.set_xticklabels(channel_fraud.index, rotation=25, ha='right')
ax.axhline(channel_fraud.mean() * 100, color='gray', linestyle='--', alpha=0.7, label='Average')
ax.legend(fontsize=10)

# --- Fraud volume by state ---
state_fraud = df_raw[df_raw['is_fraud'] == 1].groupby('state').size().sort_values(ascending=True)
ax = axes[1, 1]
ax.barh(state_fraud.index, state_fraud.values, color=COLORS['fraud'], alpha=0.8, edgecolor='white')
ax.set_title('🗺️  Fraud Transaction Count by State', fontsize=13, fontweight='bold')
ax.set_xlabel('Number of Fraud Transactions', fontsize=11)

plt.suptitle('🔍 Fraud Pattern Analysis — Nigerian Banking Transactions',
             fontsize=15, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 3.4 — CORRELATION HEATMAP
# ============================================================

numeric_cols = df_raw.select_dtypes(include=[np.number]).columns.tolist()
corr_matrix = df_raw[numeric_cols].corr()

fig, ax = plt.subplots(figsize=(14, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdYlGn',
            vmin=-1, vmax=1, center=0, linewidths=0.5, ax=ax,
            annot_kws={'size': 9})
ax.set_title('Feature Correlation Matrix\n(Lower Triangle — Symmetric Matrix)',
             fontsize=14, fontweight='bold', pad=15)
plt.tight_layout()
plt.show()

# Correlation with target
fraud_corr = corr_matrix['is_fraud'].drop('is_fraud').sort_values(key=abs, ascending=False)
print('📊 FEATURE CORRELATION WITH FRAUD LABEL (is_fraud):')
print('─' * 45)
for feat, corr_val in fraud_corr.items():
    bar = '█' * int(abs(corr_val) * 30)
    direction = '↑' if corr_val > 0 else '↓'
    print(f'  {direction} {feat:<28} {corr_val:+.4f}  {bar}')

---
# SECTION 4: Data Cleaning & Feature Engineering
---

**Data Quality Issues to address:**
1. Missing values in `merchant_category`, `distance_from_home_km`, `avg_balance_ngn`
2. Skewed numeric features (log transformation)
3. Categorical encoding for ML models
4. Feature engineering: extract new signals from existing data

In [ ]:
# ============================================================
# 4.1 — MISSING VALUE ANALYSIS
# ============================================================

missing = df_raw.isnull().sum()
missing_pct = (missing / len(df_raw) * 100).round(2)

missing_df = pd.DataFrame({
    'Missing Count': missing[missing > 0],
    'Missing %'    : missing_pct[missing > 0],
    'Strategy'     : ['Mode imputation', 'Median imputation', 'Median imputation']
})

print('🔍 MISSING VALUE REPORT:')
print('=' * 60)
print(missing_df.to_string())
print('=' * 60)
print(f'\n  Total missing cells: {missing.sum():,}')
print(f'  Overall missing  %: {missing.sum() / df_raw.size * 100:.2f}%')

# Visualize missing values
fig, ax = plt.subplots(figsize=(10, 4))
missing_plot = missing_pct[missing_pct > 0]
bars = ax.barh(missing_plot.index, missing_plot.values,
               color=COLORS['accent'], edgecolor='white', linewidth=1.5)
for bar, val in zip(bars, missing_plot.values):
    ax.text(bar.get_width() + 0.05, bar.get_y() + bar.get_height()/2,
            f'{val:.2f}%', va='center', fontsize=11)
ax.set_title('Missing Values by Feature', fontsize=13, fontweight='bold')
ax.set_xlabel('Missing Percentage (%)', fontsize=11)
ax.set_xlim(0, 6)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 4.2 — DATA CLEANING
# ============================================================

df = df_raw.copy()

print('🧹 CLEANING STEPS:')
print('─' * 50)

# Step 1: Impute missing values
df['merchant_category'] = df['merchant_category'].fillna(df['merchant_category'].mode()[0])
df['distance_from_home_km'] = df['distance_from_home_km'].fillna(df['distance_from_home_km'].median())
df['avg_balance_ngn'] = df['avg_balance_ngn'].fillna(df['avg_balance_ngn'].median())
print('  ✅ Step 1: Missing values imputed')

# Step 2: Remove impossible values
before = len(df)
df = df[df['amount_ngn'] > 0]
df = df[df['account_age_days'] >= 0]
df = df[df['txn_count_last_30d'] >= 0]
print(f'  ✅ Step 2: Removed {before - len(df)} invalid rows')

# Step 3: Cap extreme outliers (Winsorization at 99.5th percentile)
amount_cap = df['amount_ngn'].quantile(0.995)
balance_cap = df['avg_balance_ngn'].quantile(0.995)
df['amount_ngn'] = df['amount_ngn'].clip(upper=amount_cap)
df['avg_balance_ngn'] = df['avg_balance_ngn'].clip(upper=balance_cap)
print(f'  ✅ Step 3: Outliers capped — Amount ceiling: ₦{amount_cap:,.0f}')

# Step 4: Extract datetime features
df['day_of_week']    = pd.to_datetime(df['transaction_date']).dt.dayofweek
df['is_weekend']     = (df['day_of_week'] >= 5).astype(int)
df['is_night']       = ((df['transaction_hour'] >= 22) | (df['transaction_hour'] <= 5)).astype(int)
df['is_work_hours']  = ((df['transaction_hour'] >= 8) & (df['transaction_hour'] <= 17)).astype(int)
print('  ✅ Step 4: Datetime features extracted (day_of_week, is_weekend, is_night)')

# Step 5: Derived ratio features (domain-specific signals)
df['amount_to_balance_ratio'] = df['amount_ngn'] / (df['avg_balance_ngn'] + 1)
df['txn_velocity_ratio']      = df['txn_count_last_7d'] / (df['txn_count_last_30d'] / 4 + 1)
df['is_large_transaction']    = (df['amount_ngn'] > df['amount_ngn'].quantile(0.90)).astype(int)
print('  ✅ Step 5: Derived features created (amount_to_balance_ratio, txn_velocity_ratio)')

# Step 6: Log-transform skewed features
df['log_amount']           = np.log1p(df['amount_ngn'])
df['log_balance']          = np.log1p(df['avg_balance_ngn'])
df['log_distance']         = np.log1p(df['distance_from_home_km'])
df['log_account_age']      = np.log1p(df['account_age_days'])
print('  ✅ Step 6: Log transformation applied to skewed features')

print('\n📊 CLEANED DATASET:')
print(f'  Shape: {df.shape}')
print(f'  Missing values: {df.isnull().sum().sum()}')

In [ ]:
# ============================================================
# 4.3 — FEATURE ENGINEERING VISUALIZATION
# ============================================================

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

new_features = [
    ('amount_to_balance_ratio', 'Amount-to-Balance Ratio'),
    ('txn_velocity_ratio', 'Transaction Velocity Ratio'),
    ('log_distance', 'Log(Distance from Home)')
]

for ax, (feat, title) in zip(axes, new_features):
    fraud_vals  = df[df['is_fraud'] == 1][feat].clip(upper=df[feat].quantile(0.98))
    legit_vals  = df[df['is_fraud'] == 0][feat].clip(upper=df[feat].quantile(0.98))
    ax.hist(legit_vals, bins=50, alpha=0.7, color=COLORS['legit'],
            label='Legitimate', density=True)
    ax.hist(fraud_vals, bins=50, alpha=0.7, color=COLORS['fraud'],
            label='Fraudulent', density=True)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.legend(fontsize=10)
    ax.set_ylabel('Density', fontsize=10)

plt.suptitle('🔧 Engineered Features — Separability Between Fraud & Legitimate',
             fontsize=13, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("""
💡 FEATURE ENGINEERING RATIONALE:
─────────────────────────────────────────────────────────────
  amount_to_balance_ratio  → If someone with ₦5K balance tries
                             to send ₦200K — high fraud signal
  
  txn_velocity_ratio       → A sudden spike in weekly activity
                             vs. monthly baseline = account takeover
  
  log_distance             → Transactions 500km from home base
                             are unusual (card cloning / travel fraud)
─────────────────────────────────────────────────────────────
""")

---
# SECTION 5: Anomaly Detection
### Statistical Approaches to Flag Suspicious Transactions (No Labels Needed)
---

Anomaly detection is a **rule-based + statistical approach** useful when:
- Labels are not available (unsupervised)
- You want explainable rules for the compliance/fraud team
- As a pre-filter before passing to ML models

In [ ]:
# ============================================================
# 5.1 — Z-SCORE ANOMALY DETECTION
# ============================================================

from scipy import stats

# Z-score on log-transformed amount
df['zscore_log_amount']   = np.abs(stats.zscore(df['log_amount']))
df['zscore_log_balance']  = np.abs(stats.zscore(df['log_balance']))
df['zscore_velocity']     = np.abs(stats.zscore(df['txn_velocity_ratio']))
df['zscore_distance']     = np.abs(stats.zscore(df['log_distance']))

# Flag anomalies (Z > 3 means 3 standard deviations from the mean)
Z_THRESHOLD = 3.0
df['anomaly_amount']   = (df['zscore_log_amount']   > Z_THRESHOLD).astype(int)
df['anomaly_balance']  = (df['zscore_log_balance']  > Z_THRESHOLD).astype(int)
df['anomaly_velocity'] = (df['zscore_velocity']     > Z_THRESHOLD).astype(int)
df['anomaly_distance'] = (df['zscore_distance']     > Z_THRESHOLD).astype(int)

# Composite anomaly score (0–4)
df['anomaly_score'] = (df['anomaly_amount'] + df['anomaly_balance'] +
                       df['anomaly_velocity'] + df['anomaly_distance'])

print('📊 Z-SCORE ANOMALY DETECTION RESULTS:')
print('=' * 55)
for signal in ['anomaly_amount', 'anomaly_balance', 'anomaly_velocity', 'anomaly_distance']:
    flagged = df[signal].sum()
    fraud_caught = df[df[signal] == 1]['is_fraud'].sum()
    precision = fraud_caught / flagged if flagged > 0 else 0
    recall    = fraud_caught / df['is_fraud'].sum()
    print(f'  {signal:<26} | Flagged: {flagged:5,} | '
          f'Fraud caught: {fraud_caught:4,} | Precision: {precision:.1%} | Recall: {recall:.1%}')
print('=' * 55)

In [ ]:
# ============================================================
# 5.2 — RULE-BASED FRAUD SCORING (Interpretable for Business)
# ============================================================

def rule_based_fraud_score(row):
    """
    Domain-driven fraud scoring system.
    Each rule adds points — total score indicates fraud risk level.
    Rules are inspired by actual Nigerian bank fraud patterns.
    """
    score = 0
    reasons = []
    
    # Rule 1: Midnight transactions
    if row['is_night'] == 1:
        score += 2
        reasons.append('NIGHT_TXN')
    
    # Rule 2: Transaction much larger than balance
    if row['amount_to_balance_ratio'] > 2.0:
        score += 3
        reasons.append('EXCEEDS_BALANCE')
    
    # Rule 3: New recipient + large amount
    if row['is_new_recipient'] == 1 and row['amount_ngn'] > 50_000:
        score += 3
        reasons.append('NEW_RECIPIENT_LARGE')
    
    # Rule 4: Multiple failed attempts recently
    if row['failed_attempts_24h'] >= 2:
        score += 3
        reasons.append('MULTIPLE_FAILS')
    
    # Rule 5: Very far from home base
    if row['distance_from_home_km'] > 200:
        score += 2
        reasons.append('FAR_FROM_HOME')
    
    # Rule 6: Very new account + large transaction
    if row['account_age_days'] < 30 and row['amount_ngn'] > 100_000:
        score += 4
        reasons.append('NEW_ACCOUNT_LARGE_TXN')
    
    # Rule 7: Unusual activity spike
    if row['txn_velocity_ratio'] > 3.0:
        score += 2
        reasons.append('VELOCITY_SPIKE')
    
    return score, '|'.join(reasons) if reasons else 'NONE'

print('⏳ Calculating rule-based fraud scores...')
results = df.apply(rule_based_fraud_score, axis=1)
df['rule_score']   = results.apply(lambda x: x[0])
df['rule_reasons'] = results.apply(lambda x: x[1])

# Risk categories
def categorize_risk(score):
    if score == 0:   return 'LOW'
    elif score <= 3: return 'MEDIUM'
    elif score <= 6: return 'HIGH'
    else:            return 'CRITICAL'

df['risk_level'] = df['rule_score'].apply(categorize_risk)

print('\n📊 RULE-BASED RISK LEVEL DISTRIBUTION:')
print('─' * 60)
for level in ['LOW', 'MEDIUM', 'HIGH', 'CRITICAL']:
    subset = df[df['risk_level'] == level]
    fraud_rate_here = subset['is_fraud'].mean() if len(subset) > 0 else 0
    print(f'  {level:<10} | Transactions: {len(subset):7,} | '
          f'Fraud Rate: {fraud_rate_here:.1%} | '
          f'Actual Fraud: {subset["is_fraud"].sum():5,}')
print('─' * 60)

# Plot risk distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
risk_order = ['LOW', 'MEDIUM', 'HIGH', 'CRITICAL']
risk_colors = ['#2ECC71', '#F39C12', '#E67E22', '#E74C3C']

risk_counts = df['risk_level'].value_counts().reindex(risk_order, fill_value=0)
axes[0].bar(risk_counts.index, risk_counts.values, color=risk_colors, edgecolor='white', linewidth=2)
axes[0].set_title('Transaction Volume by Risk Level', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Transaction Count', fontsize=11)

risk_fraud_rates = df.groupby('risk_level')['is_fraud'].mean().reindex(risk_order, fill_value=0)
axes[1].bar(risk_fraud_rates.index, risk_fraud_rates.values * 100,
            color=risk_colors, edgecolor='white', linewidth=2)
axes[1].set_title('Fraud Rate by Risk Level (%)', fontsize=13, fontweight='bold')
axes[1].set_ylabel('Fraud Rate (%)', fontsize=11)

plt.suptitle('⚡ Rule-Based Fraud Scoring — Risk Level Analysis',
             fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

---
# SECTION 6: Machine Learning Fraud Classification
---

We train and compare **4 ML models** — escalating in complexity:

| Model | Complexity | Interpretability | Speed |
|-------|-----------|-----------------|-------|
| Logistic Regression | Low | ⭐⭐⭐⭐⭐ | Very Fast |
| Decision Tree | Low-Medium | ⭐⭐⭐⭐ | Fast |
| Random Forest | High | ⭐⭐⭐ | Medium |
| Gradient Boosting | Very High | ⭐⭐ | Slow |

> **Evaluation Metrics**: For fraud detection, **Recall** (catching as much fraud as possible) and **ROC-AUC** are more important than Accuracy.

In [ ]:
# ============================================================
# 6.1 — PREPARE FEATURES FOR ML
# ============================================================

# Select features for the model
FEATURE_COLS = [
    # Original transaction features
    'transaction_hour', 'is_weekend', 'is_night', 'is_work_hours',
    'channel', 'transaction_type',
    # Amount & balance
    'log_amount', 'log_balance', 'amount_to_balance_ratio', 'is_large_transaction',
    # Account behavior
    'log_account_age', 'txn_count_last_30d', 'txn_count_last_7d', 'txn_velocity_ratio',
    # Risk indicators
    'log_distance', 'failed_attempts_24h', 'is_new_recipient',
    # Anomaly signals
    'anomaly_score', 'rule_score'
]

TARGET_COL = 'is_fraud'

# Encode categorical features
le_channel = LabelEncoder()
le_txn     = LabelEncoder()
df['channel']          = le_channel.fit_transform(df['channel'])
df['transaction_type'] = le_txn.fit_transform(df['transaction_type'])

X = df[FEATURE_COLS].copy()
y = df[TARGET_COL].copy()

# Train / Validation / Test split (60% / 20% / 20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)

# Scale features
scaler  = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_val_sc   = scaler.transform(X_val)
X_test_sc  = scaler.transform(X_test)

print('✅ FEATURE PREPARATION COMPLETE')
print(f'   Features selected : {len(FEATURE_COLS)}')
print(f'   Training set       : {X_train.shape[0]:,} rows ({y_train.mean()*100:.1f}% fraud)')
print(f'   Validation set     : {X_val.shape[0]:,} rows ({y_val.mean()*100:.1f}% fraud)')
print(f'   Test set           : {X_test.shape[0]:,} rows ({y_test.mean()*100:.1f}% fraud)')

In [ ]:
# ============================================================
# 6.2 — TRAIN ALL MODELS
# ============================================================

print('🤖 TRAINING MODELS...')
print('─' * 40)

models = {
    'Logistic Regression' : LogisticRegression(
        class_weight='balanced', max_iter=500, random_state=42, C=0.1
    ),
    'Decision Tree'        : DecisionTreeClassifier(
        class_weight='balanced', max_depth=8, random_state=42,
        min_samples_leaf=50
    ),
    'Random Forest'        : RandomForestClassifier(
        class_weight='balanced', n_estimators=100, max_depth=10,
        random_state=42, n_jobs=-1, min_samples_leaf=20
    ),
    'Gradient Boosting'    : GradientBoostingClassifier(
        n_estimators=100, max_depth=5, learning_rate=0.1,
        random_state=42, subsample=0.8
    )
}

trained_models = {}
for name, model in models.items():
    print(f'  ⏳ Training {name}...', end='')
    model.fit(X_train_sc, y_train)
    trained_models[name] = model
    print(f' ✅')

print('\n🎉 All models trained!')

In [ ]:
# ============================================================
# 6.3 — MODEL EVALUATION & COMPARISON
# ============================================================

def evaluate_model(name, model, X, y, dataset_name='Test'):
    """Comprehensive model evaluation with business-relevant metrics."""
    y_pred      = model.predict(X)
    y_proba     = model.predict_proba(X)[:, 1]
    
    precision = precision_score(y, y_pred, zero_division=0)
    recall    = recall_score(y, y_pred)
    f1        = f1_score(y, y_pred)
    roc_auc   = roc_auc_score(y, y_proba)
    avg_prec  = average_precision_score(y, y_proba)
    
    return {
        'Model'    : name,
        'Precision': precision,
        'Recall'   : recall,
        'F1-Score' : f1,
        'ROC-AUC'  : roc_auc,
        'Avg Prec' : avg_prec,
        'y_pred'   : y_pred,
        'y_proba'  : y_proba
    }

results = {}
for name, model in trained_models.items():
    results[name] = evaluate_model(name, model, X_test_sc, y_test)

# Summary table
metrics_df = pd.DataFrame([
    {k: v for k, v in res.items() if k not in ['y_pred', 'y_proba']}
    for res in results.values()
]).set_index('Model').round(4)

print('📊 MODEL COMPARISON — TEST SET PERFORMANCE')
print('=' * 75)
print(metrics_df.to_string())
print('=' * 75)

best_model_name = metrics_df['ROC-AUC'].idxmax()
print(f'\n  🏆 Best Model (by ROC-AUC): {best_model_name} ({metrics_df.loc[best_model_name, "ROC-AUC"]:.4f})')

In [ ]:
# ============================================================
# 6.4 — CONFUSION MATRICES FOR ALL MODELS
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, (name, result) in enumerate(results.items()):
    cm = confusion_matrix(y_test, result['y_pred'])
    
    sns.heatmap(cm, annot=True, fmt=',', cmap='Blues', ax=axes[idx],
                xticklabels=['Predicted: Legit', 'Predicted: Fraud'],
                yticklabels=['Actual: Legit', 'Actual: Fraud'],
                annot_kws={'size': 14})
    
    tn, fp, fn, tp = cm.ravel()
    axes[idx].set_title(
        f'{name}\n'
        f'ROC-AUC: {result["ROC-AUC"]:.4f} | F1: {result["F1-Score"]:.4f}\n'
        f'TP: {tp:,}  FN: {fn:,}  FP: {fp:,}  TN: {tn:,}',
        fontsize=10, fontweight='bold'
    )

plt.suptitle('Confusion Matrices — All Models on Test Set',
             fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

print("""
📖 CONFUSION MATRIX GLOSSARY:
─────────────────────────────────────────────────────────────
  TP (True Positive)   → Fraud correctly flagged  ✅ (GOAL)
  TN (True Negative)   → Legit correctly approved ✅
  FP (False Positive)  → Legit wrongly blocked    ⚠️ (Bad UX)
  FN (False Negative)  → Fraud that slipped through ❌ (Costly!)

  In fraud detection: Minimizing FN = catch more fraud
  But too many FP = angry customers who can't transact!
─────────────────────────────────────────────────────────────
""")

In [ ]:
# ============================================================
# 6.5 — ROC CURVES & PRECISION-RECALL CURVES
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(18, 7))

model_colors = ['#3498DB', '#E67E22', '#2ECC71', '#9B59B6']

# --- ROC Curves ---
ax = axes[0]
for (name, result), color in zip(results.items(), model_colors):
    fpr, tpr, _ = roc_curve(y_test, result['y_proba'])
    ax.plot(fpr, tpr, color=color, linewidth=2.5,
            label=f'{name} (AUC={result["ROC-AUC"]:.4f})')

ax.plot([0,1],[0,1], 'k--', alpha=0.4, linewidth=1.5, label='Random (AUC=0.50)')
ax.fill_between([0,1],[0,1], alpha=0.05, color='gray')
ax.set_title('ROC Curves — All Models', fontsize=13, fontweight='bold')
ax.set_xlabel('False Positive Rate (FPR)', fontsize=12)
ax.set_ylabel('True Positive Rate (Recall)', fontsize=12)
ax.legend(fontsize=10, loc='lower right')
ax.set_xlim([0, 1])
ax.set_ylim([0, 1.02])

# --- Precision-Recall Curves ---
ax = axes[1]
baseline = y_test.mean()
for (name, result), color in zip(results.items(), model_colors):
    prec, rec, _ = precision_recall_curve(y_test, result['y_proba'])
    avg_p = result['Avg Prec']
    ax.plot(rec, prec, color=color, linewidth=2.5,
            label=f'{name} (AP={avg_p:.4f})')

ax.axhline(y=baseline, color='k', linestyle='--', alpha=0.5, linewidth=1.5,
           label=f'Baseline fraud rate ({baseline:.1%})')
ax.set_title('Precision–Recall Curves', fontsize=13, fontweight='bold')
ax.set_xlabel('Recall (Fraud Caught)', fontsize=12)
ax.set_ylabel('Precision (When we flag, how often right?)', fontsize=12)
ax.legend(fontsize=10, loc='upper right')
ax.set_xlim([0, 1])
ax.set_ylim([0, 1.02])

plt.suptitle('📈 Model Performance Curves — Fraud Detection',
             fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 6.6 — FEATURE IMPORTANCE (Best Model)
# ============================================================

best_model = trained_models[best_model_name]

if hasattr(best_model, 'feature_importances_'):
    importances = best_model.feature_importances_
else:
    # For Logistic Regression — use coefficient magnitudes
    importances = np.abs(best_model.coef_[0])
    importances = importances / importances.sum()

feat_imp_df = pd.DataFrame({
    'Feature': FEATURE_COLS,
    'Importance': importances
}).sort_values('Importance', ascending=True)

# Color code by feature category
def get_color(feat):
    if 'anomaly' in feat or 'rule' in feat: return '#9B59B6'
    if 'amount' in feat or 'balance' in feat: return '#E74C3C'
    if 'distance' in feat or 'night' in feat or 'hour' in feat: return '#F39C12'
    if 'velocity' in feat or 'count' in feat: return '#3498DB'
    return '#2ECC71'

colors = [get_color(f) for f in feat_imp_df['Feature']]

fig, ax = plt.subplots(figsize=(12, 9))
bars = ax.barh(feat_imp_df['Feature'], feat_imp_df['Importance'],
               color=colors, edgecolor='white', linewidth=1.5)

for bar, val in zip(bars, feat_imp_df['Importance']):
    ax.text(bar.get_width() + 0.002, bar.get_y() + bar.get_height()/2,
            f'{val:.4f}', va='center', fontsize=9)

ax.set_title(f'Feature Importances — {best_model_name}',
             fontsize=14, fontweight='bold')
ax.set_xlabel('Importance Score', fontsize=12)

# Legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#9B59B6', label='Anomaly/Rule Scores'),
    Patch(facecolor='#E74C3C', label='Amount/Balance'),
    Patch(facecolor='#F39C12', label='Time/Location'),
    Patch(facecolor='#3498DB', label='Transaction Activity'),
    Patch(facecolor='#2ECC71', label='Other'),
]
ax.legend(handles=legend_elements, fontsize=10, loc='lower right')
plt.tight_layout()
plt.show()

print(f'\n🔑 TOP 5 MOST IMPORTANT FEATURES ({best_model_name}):')
top5 = feat_imp_df.nlargest(5, 'Importance')
for _, row in top5.iterrows():
    print(f'   {row["Feature"]:<30} → {row["Importance"]:.4f}')

---
# SECTION 7: Business Impact Analysis
### Translating Model Performance into Naira (₦) Savings
---

> *"The purpose of a fraud model is not accuracy — it is profitability and customer protection."*

This section answers: **"How much money does this model actually save the bank?"**

In [ ]:
# ============================================================
# 7.1 — COST-BENEFIT ANALYSIS FRAMEWORK
# ============================================================

# Business assumptions (calibrated to Nigerian banking context)
ASSUMPTIONS = {
    'avg_fraud_loss_ngn'     : 185_000,    # Average amount lost per fraud txn
    'investigation_cost_ngn' : 5_000,      # Cost to manually investigate a flagged txn
    'customer_churn_cost_ngn': 25_000,     # Revenue lost when legit customer is wrongly blocked
    'churn_rate_if_blocked'  : 0.15,       # 15% of blocked legit customers churn
    'monthly_txn_volume'     : 1_000_000,  # Bank processes 1M txns/month
}

print('💼 BUSINESS IMPACT FRAMEWORK')
print('=' * 60)
print('  Assumptions (Nigerian Banking Context):')
for k, v in ASSUMPTIONS.items():
    if 'ngn' in k:
        print(f'    {k:<35}: ₦{v:>12,}')
    else:
        print(f'    {k:<35}: {v}')
print('=' * 60)

In [ ]:
# ============================================================
# 7.2 — COMPUTE FINANCIAL IMPACT PER MODEL
# ============================================================

def compute_business_impact(name, result, y_true, monthly_scale=20):
    """Calculate monthly financial impact of deploying a fraud model."""
    
    cm = confusion_matrix(y_true, result['y_pred'])
    tn, fp, fn, tp = cm.ravel()
    
    # Scale to monthly volume
    scale = monthly_scale
    tp_m = tp * scale
    fp_m = fp * scale
    fn_m = fn * scale
    tn_m = tn * scale
    
    A = ASSUMPTIONS
    
    # Financial calculations
    fraud_prevented        = tp_m * A['avg_fraud_loss_ngn']
    fraud_missed           = fn_m * A['avg_fraud_loss_ngn']
    investigation_costs    = (tp_m + fp_m) * A['investigation_cost_ngn']
    customer_churn_loss    = fp_m * A['churn_rate_if_blocked'] * A['customer_churn_cost_ngn']
    
    # Baseline: no model (catch nothing)
    total_fraud_without_model = (tp_m + fn_m) * A['avg_fraud_loss_ngn']
    
    # Net benefit vs. no model
    net_monthly_benefit = fraud_prevented - fraud_missed - investigation_costs - customer_churn_loss
    net_vs_no_model     = fraud_prevented - investigation_costs - customer_churn_loss
    
    return {
        'Model'                   : name,
        'Fraud Txns Caught (TP)'  : tp_m,
        'Fraud Missed (FN)'       : fn_m,
        'Legit Blocked (FP)'      : fp_m,
        'Fraud Prevented ₦'      : fraud_prevented,
        'Fraud Missed ₦'         : fraud_missed,
        'Investigation Cost ₦'   : investigation_costs,
        'Churn Loss ₦'           : customer_churn_loss,
        'Net Savings vs No Model' : net_vs_no_model,
        'ROI %'                   : net_vs_no_model / investigation_costs * 100
                                    if investigation_costs > 0 else 0
    }

impact_results = []
for name, result in results.items():
    impact = compute_business_impact(name, result, y_test, monthly_scale=20)
    impact_results.append(impact)

impact_df = pd.DataFrame(impact_results).set_index('Model')

# Format for display
print('\n💰 MONTHLY FINANCIAL IMPACT PER MODEL')
print('=' * 90)
display_cols = ['Fraud Prevented ₦', 'Fraud Missed ₦', 'Investigation Cost ₦',
                'Churn Loss ₦', 'Net Savings vs No Model', 'ROI %']

for model_name, row in impact_df[display_cols].iterrows():
    print(f'\n  📊 {model_name}')
    for col, val in row.items():
        if '₦' in col:
            print(f'     {col:<30} : ₦{val:>15,.0f}')
        elif 'ROI' in col:
            print(f'     {col:<30} : {val:>14.1f}%')
print('=' * 90)

In [ ]:
# ============================================================
# 7.3 — BUSINESS IMPACT VISUALIZATION
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# --- Stacked cost/benefit bars ---
ax = axes[0]
model_names = list(impact_df.index)
x = np.arange(len(model_names))
width = 0.35

fraud_prev_billions = impact_df['Fraud Prevented ₦'] / 1e9
total_costs_billions = (impact_df['Investigation Cost ₦'] + impact_df['Churn Loss ₦']) / 1e9

bars1 = ax.bar(x - width/2, fraud_prev_billions, width, label='Fraud Prevented',
               color=COLORS['legit'], edgecolor='white', linewidth=2)
bars2 = ax.bar(x + width/2, total_costs_billions, width, label='Total Costs',
               color=COLORS['fraud'], edgecolor='white', linewidth=2)

for bar in bars1:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f'₦{bar.get_height():.1f}B', ha='center', va='bottom', fontsize=9, fontweight='bold')
for bar in bars2:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f'₦{bar.get_height():.2f}B', ha='center', va='bottom', fontsize=9, fontweight='bold')

ax.set_title('Monthly Benefit vs Cost (₦ Billions)', fontsize=13, fontweight='bold')
ax.set_ylabel('Amount (₦ Billions)', fontsize=11)
ax.set_xticks(x)
ax.set_xticklabels([m.replace(' ', '\n') for m in model_names], fontsize=9)
ax.legend(fontsize=11)

# --- Net savings comparison ---
ax = axes[1]
net_billions = impact_df['Net Savings vs No Model'] / 1e9
bar_colors = [COLORS['legit'] if v > 0 else COLORS['fraud'] for v in net_billions]
bars = ax.bar(model_names, net_billions, color=bar_colors, edgecolor='white', linewidth=2)
for bar, val in zip(bars, net_billions):
    ax.text(bar.get_x() + bar.get_width()/2,
            bar.get_height() + (0.02 if val >= 0 else -0.05),
            f'₦{val:.2f}B', ha='center', va='bottom', fontsize=10, fontweight='bold')

ax.axhline(0, color='black', linewidth=1.5, linestyle='--')
ax.set_title('Net Monthly Savings vs No-Fraud-Detection (₦ Billions)',
             fontsize=13, fontweight='bold')
ax.set_ylabel('Net Savings (₦ Billions)', fontsize=11)
ax.set_xticklabels([m.replace(' ', '\n') for m in model_names], fontsize=9)

plt.suptitle('💰 Business Impact Analysis — Monthly Financial Value of Fraud Detection Models',
             fontsize=13, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

---
# SECTION 8: Threshold Optimization
### Finding the Optimal Decision Threshold for the Best Model
---

By default, ML models classify fraud when probability > 0.5. But we can tune this threshold to:
- **Lower threshold** → Catch more fraud (higher Recall) but flag more legit transactions (more FP)
- **Higher threshold** → Fewer false alarms but miss more fraud

In [ ]:
# ============================================================
# 8.1 — THRESHOLD OPTIMIZATION
# ============================================================

best_model_result = results[best_model_name]
y_proba_best      = best_model_result['y_proba']

thresholds  = np.arange(0.05, 0.95, 0.01)
precision_t = []
recall_t    = []
f1_t        = []
net_save_t  = []

for thr in thresholds:
    y_pred_t = (y_proba_best >= thr).astype(int)
    p = precision_score(y_test, y_pred_t, zero_division=0)
    r = recall_score(y_test, y_pred_t, zero_division=0)
    f = f1_score(y_test, y_pred_t, zero_division=0)
    
    # Net savings at this threshold
    cm_t = confusion_matrix(y_test, y_pred_t)
    tn_t, fp_t, fn_t, tp_t = cm_t.ravel()
    scale = 20
    net = ((tp_t * scale * ASSUMPTIONS['avg_fraud_loss_ngn'])
         - ((tp_t + fp_t) * scale * ASSUMPTIONS['investigation_cost_ngn'])
         - (fp_t * scale * ASSUMPTIONS['churn_rate_if_blocked']
            * ASSUMPTIONS['customer_churn_cost_ngn'])) / 1e9
    
    precision_t.append(p)
    recall_t.append(r)
    f1_t.append(f)
    net_save_t.append(net)

optimal_idx  = np.argmax(f1_t)
optimal_thr  = thresholds[optimal_idx]
best_net_idx = np.argmax(net_save_t)
best_net_thr = thresholds[best_net_idx]

fig, axes = plt.subplots(1, 2, figsize=(18, 6))

ax = axes[0]
ax.plot(thresholds, precision_t, color='#3498DB', linewidth=2.5, label='Precision')
ax.plot(thresholds, recall_t,    color='#E74C3C', linewidth=2.5, label='Recall')
ax.plot(thresholds, f1_t,        color='#2ECC71', linewidth=2.5, label='F1-Score')
ax.axvline(optimal_thr, color='#F39C12', linestyle='--', linewidth=2,
           label=f'Best F1 Threshold = {optimal_thr:.2f}')
ax.axvline(0.50, color='gray', linestyle=':', linewidth=1.5, label='Default = 0.50')
ax.set_title('Metrics vs Decision Threshold', fontsize=13, fontweight='bold')
ax.set_xlabel('Threshold', fontsize=11)
ax.set_ylabel('Score', fontsize=11)
ax.legend(fontsize=10)

ax = axes[1]
ax.plot(thresholds, net_save_t, color='#9B59B6', linewidth=3)
ax.fill_between(thresholds, net_save_t, 0,
                where=np.array(net_save_t) > 0, alpha=0.2, color='#2ECC71')
ax.axvline(best_net_thr, color='#F39C12', linestyle='--', linewidth=2,
           label=f'Max Profit Threshold = {best_net_thr:.2f}')
ax.axvline(0.50, color='gray', linestyle=':', linewidth=1.5, label='Default = 0.50')
ax.set_title('Monthly Net Savings (₦B) vs Decision Threshold', fontsize=13, fontweight='bold')
ax.set_xlabel('Threshold', fontsize=11)
ax.set_ylabel('Monthly Net Savings (₦ Billions)', fontsize=11)
ax.legend(fontsize=10)

plt.suptitle(f'⚙️  Threshold Optimization — {best_model_name}',
             fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

print(f"""
📌 THRESHOLD RECOMMENDATIONS:
─────────────────────────────────────────────────────────────
  Default threshold (0.50):
    F1-Score: {f1_t[49]:.4f} | Net Savings: ₦{net_save_t[49]:.2f}B/month

  Optimal for F1 ({optimal_thr:.2f}):
    F1-Score: {f1_t[optimal_idx]:.4f} | Net Savings: ₦{net_save_t[optimal_idx]:.2f}B/month

  Optimal for profit ({best_net_thr:.2f}):
    F1-Score: {f1_t[best_net_idx]:.4f} | Net Savings: ₦{net_save_t[best_net_idx]:.2f}B/month

  💡 RECOMMENDATION: Use threshold {best_net_thr:.2f} for production
     deployment to maximize financial return.
─────────────────────────────────────────────────────────────
""")

---
# SECTION 9: Executive Summary & Conclusions
---

In [ ]:
# ============================================================
# 9.1 — FINAL SUMMARY DASHBOARD
# ============================================================

best_result = results[best_model_name]
best_impact = [r for r in impact_results if r['Model'] == best_model_name][0]

print("""╔══════════════════════════════════════════════════════════════════╗
║    NIGERIAN BANK FRAUD DETECTION — EXECUTIVE SUMMARY REPORT    ║
╚══════════════════════════════════════════════════════════════════╝""")

print(f"""
📋 PROJECT OVERVIEW
  Dataset      : 50,000 simulated Nigerian bank transactions (2023)
  Fraud Rate   : 2.5% ({df['is_fraud'].sum():,} fraudulent transactions)
  Total ₦ at risk (Test): ₦{df_raw[df_raw['is_fraud']==1]['amount_ngn'].sum():,.0f}

────────────────────────────────────────────────────────────────────
🤖 BEST PERFORMING MODEL: {best_model_name}
────────────────────────────────────────────────────────────────────
  ROC-AUC Score    : {best_result['ROC-AUC']:.4f}  (1.0 = perfect, 0.5 = random)
  Precision        : {best_result['Precision']:.4f}  (When we flag fraud, we're correct {best_result['Precision']*100:.1f}% of the time)
  Recall           : {best_result['Recall']:.4f}  (We catch {best_result['Recall']*100:.1f}% of all actual fraud)
  F1-Score         : {best_result['F1-Score']:.4f}  (Harmonic mean of Precision & Recall)

────────────────────────────────────────────────────────────────────
💰 MONTHLY FINANCIAL IMPACT (Scaled to 1M transactions/month)
────────────────────────────────────────────────────────────────────
  Fraud Transactions Caught  : {best_impact['Fraud Txns Caught (TP)']:>10,.0f}
  Fraud Transactions Missed  : {best_impact['Fraud Missed (FN)']:>10,.0f}
  Legitimate Transactions Blocked (FP): {best_impact['Legit Blocked (FP)']:>5,.0f}

  Fraud Losses Prevented     : ₦{best_impact['Fraud Prevented ₦']:>15,.0f}
  Investigation Costs        : ₦{best_impact['Investigation Cost ₦']:>15,.0f}
  Customer Churn Loss        : ₦{best_impact['Churn Loss ₦']:>15,.0f}
  ─────────────────────────────────────────────────
  NET MONTHLY SAVINGS        : ₦{best_impact['Net Savings vs No Model']:>15,.0f}
  ANNUAL SAVINGS             : ₦{best_impact['Net Savings vs No Model']*12:>15,.0f}
  ROI                        : {best_impact['ROI %']:>14.1f}%

────────────────────────────────────────────────────────────────────
🎯 KEY FRAUD SIGNALS DISCOVERED (Top 5 Predictors)
────────────────────────────────────────────────────────────────────""")

for i, (_, row) in enumerate(top5.iterrows(), 1):
    print(f"  {i}. {row['Feature']:<32} → Importance: {row['Importance']:.4f}")

print(f"""
────────────────────────────────────────────────────────────────────
⚠️  FRAUD RISK PATTERNS IDENTIFIED
────────────────────────────────────────────────────────────────────
  🌙 Late Night (22:00–06:00): Higher fraud rate than daytime
  💸 Large Amounts vs Balance: Key indicator of account takeover
  🏃 Transaction Velocity:    Unusual spikes = credential stuffing
  🌍 Distance from Home:      Geolocation anomalies flag card fraud
  🆕 New Accounts + Big Txns: Common money mule pattern in Nigeria
  🔄 New Recipients:          80% of fraud goes to new accounts

────────────────────────────────────────────────────────────────────
🚀 DEPLOYMENT RECOMMENDATIONS
────────────────────────────────────────────────────────────────────
  1. PRODUCTION THRESHOLD: Use {best_net_thr:.2f} (maximizes profit vs. 0.50 default)
  2. REAL-TIME SCORING: Integrate with NIBSS for sub-100ms decisions
  3. MONITORING: Track model drift monthly — fraud patterns evolve
  4. HUMAN REVIEW: Route HIGH/CRITICAL risk transactions to fraud team
  5. FEEDBACK LOOP: Retrain quarterly with confirmed fraud labels
  6. EXPLAINABILITY: Use rule reasons to communicate blocks to customers
────────────────────────────────────────────────────────────────────
""")

In [ ]:
# ============================================================
# 9.2 — FINAL COMPREHENSIVE DASHBOARD
# ============================================================

fig = plt.figure(figsize=(20, 14))
fig.suptitle('🏦 Nigerian Bank Fraud Detection — Final Dashboard',
             fontsize=18, fontweight='bold', y=0.98)

# Model comparison bar chart
ax1 = fig.add_subplot(3, 4, (1, 2))
metric_names = ['Precision', 'Recall', 'F1-Score', 'ROC-AUC']
x_m = np.arange(len(metric_names))
bar_width = 0.18
model_colors_list = ['#3498DB', '#E67E22', '#2ECC71', '#9B59B6']
for i, (mname, mresult) in enumerate(results.items()):
    vals = [mresult['Precision'], mresult['Recall'], mresult['F1-Score'], mresult['ROC-AUC']]
    ax1.bar(x_m + i * bar_width, vals, bar_width,
            label=mname, color=model_colors_list[i], alpha=0.85, edgecolor='white')
ax1.set_xticks(x_m + bar_width * 1.5)
ax1.set_xticklabels(metric_names, fontsize=10)
ax1.set_title('Model Performance Comparison', fontsize=11, fontweight='bold')
ax1.legend(fontsize=8, loc='lower right')
ax1.set_ylim(0, 1.15)
ax1.axhline(1.0, color='gray', linestyle='--', alpha=0.5, linewidth=1)

# ROC curve (best model)
ax2 = fig.add_subplot(3, 4, (3, 4))
fpr_b, tpr_b, _ = roc_curve(y_test, best_result['y_proba'])
ax2.plot(fpr_b, tpr_b, color='#E74C3C', linewidth=3,
         label=f'{best_model_name}\nAUC={best_result["ROC-AUC"]:.4f}')
ax2.plot([0,1],[0,1], 'k--', alpha=0.4)
ax2.fill_between(fpr_b, tpr_b, alpha=0.15, color='#E74C3C')
ax2.set_title(f'ROC Curve — {best_model_name}', fontsize=11, fontweight='bold')
ax2.set_xlabel('FPR', fontsize=10)
ax2.set_ylabel('TPR', fontsize=10)
ax2.legend(fontsize=10)

# Confusion matrix (best model)
ax3 = fig.add_subplot(3, 4, 5)
cm_best = confusion_matrix(y_test, best_result['y_pred'])
sns.heatmap(cm_best, annot=True, fmt=',', cmap='Blues', ax=ax3,
            xticklabels=['Legit', 'Fraud'], yticklabels=['Legit', 'Fraud'],
            annot_kws={'size': 11})
ax3.set_title(f'Confusion Matrix\n{best_model_name}', fontsize=10, fontweight='bold')

# Feature importance (top 10)
ax4 = fig.add_subplot(3, 4, (6, 7))
top10 = feat_imp_df.tail(10)
ax4.barh(top10['Feature'], top10['Importance'],
         color=[get_color(f) for f in top10['Feature']], edgecolor='white')
ax4.set_title('Top 10 Feature Importances', fontsize=10, fontweight='bold')
ax4.set_xlabel('Importance', fontsize=9)

# Risk level fraud rates
ax5 = fig.add_subplot(3, 4, 8)
risk_rates = df.groupby('risk_level')['is_fraud'].mean().reindex(risk_order, fill_value=0) * 100
ax5.bar(risk_rates.index, risk_rates.values, color=['#2ECC71','#F39C12','#E67E22','#E74C3C'],
        edgecolor='white', linewidth=1.5)
ax5.set_title('Fraud Rate by Risk Level', fontsize=10, fontweight='bold')
ax5.set_ylabel('Fraud %', fontsize=9)

# Fraud by hour
ax6 = fig.add_subplot(3, 4, (9, 10))
hourly_rate = df.groupby('transaction_hour')['is_fraud'].mean() * 100
ax6.fill_between(hourly_rate.index, hourly_rate.values, alpha=0.6, color='#E74C3C')
ax6.plot(hourly_rate.index, hourly_rate.values, color='#E74C3C', linewidth=2)
ax6.axvspan(-0.5, 5.5, alpha=0.1, color='red')
ax6.axvspan(21.5, 23.5, alpha=0.1, color='red')
ax6.set_title('Fraud Rate by Hour of Day', fontsize=10, fontweight='bold')
ax6.set_xlabel('Hour', fontsize=9)
ax6.set_ylabel('Fraud %', fontsize=9)
ax6.set_xticks(range(0, 24, 3))

# Monthly savings comparison
ax7 = fig.add_subplot(3, 4, (11, 12))
net_savings = [r['Net Savings vs No Model'] / 1e9 for r in impact_results]
model_labels = [r['Model'].replace(' ', '\n') for r in impact_results]
bar_c = [COLORS['legit'] if v > 0 else COLORS['fraud'] for v in net_savings]
ax7.bar(model_labels, net_savings, color=bar_c, edgecolor='white', linewidth=1.5)
ax7.set_title('Monthly Net Savings (₦B)', fontsize=10, fontweight='bold')
ax7.set_ylabel('₦ Billions/month', fontsize=9)
ax7.axhline(0, color='black', linewidth=1.2)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig('/home/claude/fraud_detection_dashboard.png', dpi=150, bbox_inches='tight')
plt.show()
print('✅ Dashboard saved!')

---
## 📚 Skills Demonstrated in This Project

| Category | Skills |
|----------|--------|
| **Data Engineering** | Simulating realistic domain data, handling missing values, feature engineering, Winsorization, log transformation |
| **EDA** | Distribution analysis, correlation matrices, temporal pattern analysis, segment comparison |
| **Anomaly Detection** | Z-score anomaly detection, rule-based fraud scoring, composite risk scoring |
| **Machine Learning** | Logistic Regression, Decision Tree, Random Forest, Gradient Boosting, Stratified Train/Test splits, class imbalance handling |
| **Model Evaluation** | ROC-AUC, Precision-Recall, Confusion Matrix, F1-Score, threshold optimization |
| **Business Impact** | Cost-benefit analysis, ROI calculation, financial modeling in Nigerian Naira |
| **Communication** | Executive summary, visualization dashboards, business recommendations |
| **Domain Knowledge** | NIBSS payment ecosystem, Nigerian banking patterns, FITC fraud statistics |

---

## 🔗 Next Steps & Extensions

1. **XGBoost / LightGBM** — More powerful gradient boosting for higher AUC
2. **SMOTE Oversampling** — Synthetic data generation to address class imbalance
3. **Network Analysis** — Graph-based fraud ring detection (Neo4j)
4. **Real-time API** — Flask/FastAPI endpoint for live transaction scoring
5. **Explainability** — SHAP values for individual transaction explanations
6. **Time-Series** — Sequential fraud detection with LSTM networks
7. **MLOps** — Model monitoring, drift detection, automated retraining pipeline

---

*Built with ❤️ for the Nigerian FinTech ecosystem | Python 3.x | Scikit-learn*